In [ ]:
# TODO: make sure that your machine has CPU and storage. Fasterq-dump uses 6 threads as default

In [ ]:
# !mkdir /home/jupyter/eila/

In [ ]:
# !wget -P /home/jupyter/eila/ https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/3.0.5/sratoolkit.3.0.5-ubuntu64.tar.gz

In [ ]:
# !tar -xvzf /home/jupyter/eila/sratoolkit.3.0.5-ubuntu64.tar.gz -C /home/jupyter/eila/

In [ ]:
# !ls /home/jupyter/eila/sratoolkit.3.0.5-ubuntu64/bin

In [ ]:
# this is the execution from the command line

# jupyter nbconvert --execute --clear-output download_gse_data.ipynb --stdout

In [ ]:
# !pip install biopython

In [ ]:
## TODO: run the below command once /home/jupyter/.ncbi/user-settings.mkfg -
# !/home/jupyter/eila/sratoolkit.3.0.5-ubuntu64/bin/vdb-config --set "/LIBS/GUID/DEFAULT/caching"=false

In [ ]:
import subprocess
from Bio import Entrez
import os
import xml.etree.ElementTree as ET
import shutil

def get_srr_accession_list(gsm_id):
    # Search for the GSM ID in the SRA database
    handle = Entrez.esearch(db="sra", term=gsm_id)
    record = Entrez.read(handle)

    # Get the list of SRR accessions from the search results
    srr_accessions = record['IdList']

    # Fetch the SRR records to get their metadata
    handle = Entrez.efetch(db="sra", id=srr_accessions, retmode="xml")
    xml_data = handle.read()

    # Parse the XML data using ElementTree
    root = ET.fromstring(xml_data)

    # Extract the SRR from the parsed XML
    srr_accessions = []
    for run_element in root.findall(".//RUN"):
      srr = run_element.attrib
      srr_accessions.append(srr['accession'])
      print("srr_accessions is {}".format(srr_accessions))

    return srr_accessions

# Provide your email address to the Entrez API
# TODO1: change it to your email
Entrez.email = "name@example.edu"
# TODO2:
# Set your GSE ID. Line 44 on gSheet was downloaded
gse_id_list = ["GSE175621",
               "GSE214979", #subset of the single cell
               "GSE147672"]

# TODO3:
# Set your project_name:
project_name = ["clint-miller_human_coronary-artery_scatac", 
                "BrainAlzheimer10XMultiome",
                "HealthyBrainAdult_snATAC"]
                

for index, gse_id in enumerate(gse_id_list):
    
  # Search for the GSE ID in the GEO database
  handle = Entrez.esearch(db="gds", term=gse_id)
  record = Entrez.read(handle)

  # Fetch the GSE record using the retrieved ID
  gse_record = record["IdList"][0]
  handle = Entrez.esummary(db="gds", id=gse_record)
  gse_summary = Entrez.read(handle)[0]

  # Extract the GSM IDs from the GSE record
  gsm_ids_objects = gse_summary["Samples"]

  gsm_ids = [gsm_id["Accession"] for gsm_id in gsm_ids_objects]
  # Print the list of GSM IDs
  print("gsm_ids is {}".format(gsm_ids))

  # Specify the cloud storage directory path
  # TODO: if you are intereted in coping the data to gs bucket
  cloud_directory = "gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/FASTQ/{}/".format(project_name[index])

  # Iterate over the GSM IDs
  for gsm_id in gsm_ids:
    print('gsm_id is {}'.format(gsm_id))
    
    
    # TODO: check this for single cell explanation and select the right flags: https://github.com/ncbi/sra-tools/issues/418
    srr_accessions = get_srr_accession_list(gsm_id)
    print('srr_accessions is {}'.format(srr_accessions))
    
    
    for srr in srr_accessions:
      cloud_directory_gsm = os.path.join(cloud_directory,gsm_id)
      print('cloud_directory_gsm is {}'.format(cloud_directory_gsm))
        
      cloud_directory_gsm_srr = os.path.join(cloud_directory_gsm,srr)
      print('cloud_directory_gsm_srr is {}'.format(cloud_directory_gsm_srr))
      ls_output_gs_srr = !gsutil ls $cloud_directory_gsm_srr
      print('ls_output_gs_srr is {}'.format(ls_output_gs_srr))
      if "CommandException" in ls_output_gs_srr[0]:
        local_directory_srr = os.path.join("/home/jupyter/eila/",gse_id,gsm_id,srr)
        print('local_directory_srr is {}'.format(local_directory_srr))
        os.makedirs(local_directory_srr, exist_ok=True)
#         fasterq-dump does not take multiple accessions, just one
        cmd = ["/home/jupyter/eila/sratoolkit.3.0.5-ubuntu64/bin/fasterq-dump",srr,"--split-files","--progress","--include-technical","--outdir", local_directory_srr]
        print("Download cmd is {}".format(cmd))
        subprocess.run(cmd)
        # TODO: if you are not interested in copying to GS. remove these lines
        print("calling gsutil -m cp -r {} {}".format(local_directory_srr,cloud_directory_gsm))
        !gsutil -m cp -r $local_directory_srr $cloud_directory_gsm
        print("finished gsutil")
        # TODO: if you are not interested in deleting the local copy after copying.remove these lines
        shutil.rmtree(local_directory_srr,ignore_errors=True)
        print("deleted {}".format(local_directory_srr))


In [38]:
!ls -lta /home/jupyter/eila/GSE175621/GSM5342819/SRR14664711


total 59925560
-rw-rw-r-- 1 jupyter users 14608989848 Jun 29 22:32 SRR14664711_1.fastq
-rw-rw-r-- 1 jupyter users 14760152872 Jun 29 22:32 SRR14664711_3.fastq
-rw-rw-r-- 1 jupyter users 16056594760 Jun 29 22:32 SRR14664711_2.fastq
-rw-rw-r-- 1 jupyter users 15937970488 Jun 29 22:32 SRR14664711_4.fastq
drwxrwxr-x 2 jupyter users        4096 Jun 29 21:58 .
drwxrwxr-x 3 jupyter users        4096 Jun 29 21:02 ..


In [41]:
!ls -lta /home/jupyter/eila/GSE214979/GSM6619558/SRR21834811


total 74304372
-rw-rw-r-- 1 jupyter users 28903203168 Jun 29 22:40 SRR21834811_4.fastq
-rw-rw-r-- 1 jupyter users 17864631656 Jun 29 22:18 SRR21834811_3.fastq
-rw-rw-r-- 1 jupyter users 14659885088 Jun 29 22:10 SRR21834811_2.fastq
-rw-rw-r-- 1 jupyter users 14659885088 Jun 29 22:10 SRR21834811_1.fastq
drwxrwxr-x 2 jupyter users        4096 Jun 29 21:25 .
drwxrwxr-x 3 jupyter users        4096 Jun 29 20:58 ..


In [ ]:
# gse_id_list[-]

In [34]:
!ls fasterq.tmp.* | wc -l

84


In [37]:
!ls fasterq.tmp.* | wc -l

83
